# 1. Packages

In [33]:
import pandas as pd
import numpy as np

# 2. Data information

In [34]:
df = pd.read_csv (r'C:\Users\juand\OneDrive\Escritorio\github_projects\ICE-Video-Game\datasets\games.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


# 3. Problem framing
**Business goal:** Identify which platform/genre combinations and geographic regions should be prioritized to maximize sales over the next 2–3 years.
**Stakeholder questions:**
- Which platforms and genres generate the most total and regional sales?
- How are sales trends evolving over time overall and per platform?
- What platform/genre/publisher mixes offer the strongest return on investment?
- Are there emerging opportunities (new consoles, niche genres, specific regions) showing accelerating growth?
**KPIs to track:** global sales, regional sales (NA/EU/JP/Other), YoY growth, platform share, genre performance, release cadence.
**Decision context:** The analysis should inform launch planning and catalog optimization by tying each insight to a numeric metric or visualization that stakeholders can act on.


# 4. Data cleaning

In [35]:
# Columns standardization and cleaning
# ------------------------------
df.columns = df.columns.str.lower()

# Normalize release year and ensure realistic range
df = df.dropna(subset=['year_of_release']).copy()
df['year_of_release'] = (
    pd.to_numeric(df['year_of_release'], errors='coerce')
    .clip(lower=1980, upper=2016)
    .astype(int)
)

# Normalize score fields
df['user_score'] = (
    df['user_score']
      .astype(str)
      .str.strip()
      .str.lower()
      .replace({'tbd': np.nan})
)
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce')

df['critic_score'] = pd.to_numeric(df['critic_score'], errors='coerce') / 10

# Standardize ESRB ratings
df['rating'] = (
    df['rating']
      .astype(str)
      .str.strip()
      .str.lower()
      .replace({'rp': np.nan})
      .fillna('unknown')
)

# Boolean indicators for critic and user score availability
df['critic_score_missing'] = df['critic_score'].isna()
df['user_score_missing'] = df['user_score'].isna()

# Remove games without names/genres and deduplicate by core identifiers
df = df.dropna(subset=['name', 'genre']).copy()
df = df.drop_duplicates(subset=['name', 'platform', 'year_of_release'], keep='first')

# Platform normalization and DS launch sanity check
df['platform'] = df['platform'].astype(str).str.strip().str.upper()
df = df[~((df['platform'] == 'DS') & (df['year_of_release'] < 2004))]

# Standardize categorical text fields
text_cols = ['name', 'genre']
for column in text_cols:
    df[column] = (
        df[column]
          .astype(str)
          .str.strip()
          .str.lower()
    )

category_cols = text_cols + ['platform', 'rating']
for column in category_cols:
    df[column] = df[column].astype('category')

# Create total sales column and drop rows without sales
df['total_sales'] = df[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)
df = df[df['total_sales'] > 0].copy()


In [36]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 16440 entries, 0 to 16714
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   name                  16440 non-null  category
 1   platform              16440 non-null  category
 2   year_of_release       16440 non-null  int64   
 3   genre                 16440 non-null  category
 4   na_sales              16440 non-null  float64 
 5   eu_sales              16440 non-null  float64 
 6   jp_sales              16440 non-null  float64 
 7   other_sales           16440 non-null  float64 
 8   critic_score          7982 non-null   float64 
 9   user_score            7462 non-null   float64 
 10  rating                16440 non-null  category
 11  critic_score_missing  16440 non-null  bool    
 12  user_score_missing    16440 non-null  bool    
 13  total_sales           16440 non-null  float64 
dtypes: bool(2), category(4), float64(7), int64(1)
memory usage:

# 5. Data profiling & quality
This section validates the cleaned dataset by summarizing coverage, missing values, and key distribution checks before EDA.


In [37]:
# High-level structure and uniqueness checks
n_rows, n_cols = df.shape
duplicate_rows = df.duplicated(subset=['name', 'platform', 'year_of_release']).sum()
profile = {
    'rows': f"{n_rows:,}",
    'columns': n_cols,
    'year_range': f"{df['year_of_release'].min()} - {df['year_of_release'].max()}",
    'platforms': df['platform'].nunique(),
    'genres': df['genre'].nunique(),
    'ratings': df['rating'].nunique()
}
for key, value in profile.items():
    print(f"{key.title()}: {value}")
print(f"Duplicate name/platform/year rows remaining: {duplicate_rows}")
print('Top 5 platforms by game count:')
print(df['platform'].value_counts().head())


Rows: 16,440
Columns: 14
Year_Range: 1980 - 2016
Platforms: 31
Genres: 12
Ratings: 9
Duplicate name/platform/year rows remaining: 0
Top 5 platforms by game count:
platform
PS2     2126
DS      2120
PS3     1304
WII     1286
X360    1232
Name: count, dtype: int64


In [38]:
# Missing data summary
missing_summary = (
    df.isna().sum()
    .to_frame(name='missing_count')
    .assign(missing_pct=lambda x: (x['missing_count'] / len(df)).round(4))
    .sort_values('missing_pct', ascending=False)
)
missing_summary.head(10)


,missing_count,missing_pct
user_score,8978,0.5461
critic_score,8458,0.5145
year_of_release,0,0.0000
genre,0,0.0000
name,0,0.0000
platform,0,0.0000
eu_sales,0,0.0000
na_sales,0,0.0000
other_sales,0,0.0000
jp_sales,0,0.0000


In [39]:
# Numeric distribution check for sales and release year
sales_cols = ['na_sales', 'eu_sales', 'jp_sales', 'other_sales', 'total_sales']
df[sales_cols + ['year_of_release']].describe().T


,count,mean,std,min,25%,50%,75%,max
na_sales,16440.0,0.264077,0.818467,0.00,0.00,0.08,0.24,41.36
eu_sales,16440.0,0.145965,0.506773,0.00,0.00,0.02,0.11,28.96
jp_sales,16440.0,0.078505,0.311136,0.00,0.00,0.00,0.04,10.22
other_sales,16440.0,0.047605,0.188027,0.00,0.00,0.01,0.03,10.57
total_sales,16440.0,0.536152,1.558954,0.01,0.06,0.17,0.47,82.54
year_of_release,16440.0,2006.487044,5.873669,1980.00,2003.00,2007.00,2010.00,2016.00


### Data quality notes
- Critic scores are available for roughly 48% of games and user scores for ~45%, so visualizations should highlight missingness.
- ESRB ratings were standardized to lowercase strings with an 'unknown' bucket capturing previously missing labels.
- Duplicates were removed based on name/platform/year; remaining DS titles before 2004 were filtered to avoid pre-launch noise.
- Sales figures now include a `total_sales` metric and rows with zero reported sales were removed to focus on meaningful releases.
